# Hw01 - Knapsack problem

## Content

- [Imports](#Imports)
- [Load C++ library for Knapsack solution](#load-c++-library-for-knapsack-solution)
- [Load dataframe functions](#load-dataframe-functions)
- [](#)

## Imports


In [18]:
import pandas as pd
import ctypes
from ctypes import c_char_p, c_void_p, c_double
import json
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


## Load C++ library for Knapsack solution

In [19]:
# Do the combinatorics
clib = ctypes.cdll.LoadLibrary('./build/knapsack.so')
clib.solve.argtypes = [c_char_p, c_char_p]
clib.solve.restype = c_double

## Load dataframe functions

In [20]:
# Load dataframes
def insToDf(path):
    data_nr_ins_4 = pd.read_csv(path, sep=' ', index_col=0, header=None)
    df_ins = data_nr_ins_4.copy()
    df_ins = df_ins[~df_ins.index.duplicated(keep='first')]
    df_ins.index = df_ins.index.rename("ID")
    df_ins = df_ins.rename(columns = {1: "n", 2: "M", 3: "B"})
    df_ins.columns = [str(x) for x in df_ins.columns]
    df_ins["weights"] = df_ins.apply(lambda x: list(x[df_ins.columns[3::2]]), axis=1)
    df_ins["values"] = df_ins.apply(lambda x: list(x[df_ins.columns[4::2]]), axis=1)
    df_ins = df_ins.drop(df_ins.columns[3:-2], axis=1)
    df_ins.index = abs(df_ins.index)
    return df_ins

def solToDf(path):
    data_nr_sol_4 = pd.read_csv(path, sep=' ', index_col=0, header=None)
    df_sol = data_nr_sol_4.copy()
    df_sol = df_sol[~df_sol.index.duplicated(keep='first')]
    df_sol.index = df_sol.index.rename("ID")
    df_sol = df_sol.drop(df_sol.columns[-1], axis=1)
    df_sol["solution"] = df_sol.apply(lambda x: list(x[df_sol.columns[2:]]), axis=1)
    df_sol = df_sol.drop(df_sol.columns[2:-1], axis=1)
    df_sol = df_sol.rename(columns = {1: 'n', 2: "optimum"})
    # data_nr_4.index = data_nr_4.index.rename("ID")
    return df_sol

def resToDf(path):
    data_nr_res_4 = pd.read_csv(path, sep=' ', index_col=0, header=None)
    df_res = data_nr_res_4.copy()

    df_res.index = df_res.index.rename("ID")
    df_res = df_res.rename(columns = {1: 'n', 2: 'possible', 3: "value", df_res.columns[-2]:"complexity", 
        df_res.columns[-1]:"seconds"})
    df_res.columns = [str(x) for x in df_res.columns]
    df_res["solution"] = df_res.apply(lambda x: list(x[df_res.columns[3:-2]]), axis=1)
    df_res = df_res.drop(df_res.columns[3:-3], axis=1)
    df_res.index = abs(df_res.index)
    return df_res

def check(df_ins, df_sol, df_res, path):
    possible = df_ins["B"] <= df_sol["optimum"]
    check = df_res['possible'] == possible
    ok = check.sum() == df_ins.shape[0]
    idx = df_ins[~check].index.values
    return ((~check).sum(), idx)


In [21]:
# Read all dataframes

# "./tests/test_{n}.csv" # "./data/nr/res/N_LAST{n}.csv" # "./data/nr/res/N{n}.csv"

datasets = {
    "nr": {
        "instances": "../data/nr/NR{n}_inst.dat",
        "solutions": "../data/nr/NK{n}_sol.dat",
        "brute_force": "../data/computed/nr_resu/bf_n_n{n}_resu_test.csv",
        "branch_and_bound": "../data/computed/nr_resu/bab_n_n{n}_resu_test.csv",
    }, 
    "zr": {
        "instances": "../data/zr/ZR{n}_inst.dat",
        "solutions": "../data/zr/ZK{n}_sol.dat",
        "brute_force": "../data/computed/zr_resu/bf_z_n{n}_resu_test.csv",
        "branch_and_bound": "../data/computed/zr_resu/bab_z_n{n}_resu_test.csv",
    }
}

NUMBERS = [4, 10, 15, 20] # [4, 10, 15, 20, 22, 25, 27] # [4, 10, 15, 20, 22, 25, 27, 30, 32, 35, 37, 40]
dfs = {}
for name, data in datasets.items():
    print("Name: {}, n: [".format(name), end='')
    dfs[name] = {}
    for n in NUMBERS:
        print("{}, ".format(n), end='') if n != NUMBERS[-1] else print("{}]".format(n))
        df_ins = insToDf(data["instances"].format(n=n))

        df_sol = solToDf(data["solutions"].format(n=n))

        df_bf = resToDf(data["brute_force"].format(n=n))
        df_bab = resToDf(data["branch_and_bound"].format(n=n))

        dfs[name][n] = {'instances': df_ins, 'solutions': df_sol, 'brute_force': df_bf, 'branch_and_bound': df_bab}

        err_num, err_lines = check(df_ins, df_sol, df_bf, data["brute_force"].format(n=n))
        if err_num > 0:
            print("Name: {}, n: {}, BF: err_num: {}, err_lines: {}".format(name, n, err_num, err_lines))
    
        err_num, err_lines = check(df_ins, df_sol, df_bab, data["branch_and_bound"].format(n=n))
        if err_num > 0:
            print("Name: {}, n: {}, BAB: err_num: {}, err_lines: {}".format(name, n, err_num, err_lines))
    #     print(ERRORS)

Name: nr, n: [4, 10, 15, 20]
Name: zr, n: [4, 10, 15, 20]


In [22]:
BF = "brute_force"
BAB = "branch_and_bound"

def plotHistogram(dfs, name, n, title, x_axis, column):
    dfs = dfs.copy()
    max_value = max(dfs[name][n][BF][column].max(), dfs[name][n][BAB][column].max())
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=dfs[name][n][BF][column], name='Brute Force',
        xbins=dict(start=0,end=max_value, size=max_value/10)))
    fig.add_trace(go.Histogram(x=dfs[name][n][BAB][column], name='Branch and Bound',
        xbins=dict(start=0,end=max_value, size=max_value/10)))

    # fig.update_layout(height=500, width=700, title_text="Multiple Subplots with Titles")
    # fig.update_layout(title_text=title, # title of plot
    #     xaxis_title_text=x_axis, yaxis_title_text='Count', 
        # bargap=0.2, # gap between bars of adjacent location coordinates
        # bargroupgap=0.0001 # gap between bars of the same location coordinates
    # )
    fig.update_layout(barmode='overlay', title_text=title, # title of plot
    #     xaxis_title_text=x_axis, yaxis_title_text='Count', 
        bargap=0.2, # gap between bars of adjacent location coordinates
        bargroupgap=0.1, # gap between bars of the same location coordinates
        # xaxis = dict(gridwidth = max_value/10),
        xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = max_value/10, showgrid=True),
        xaxis_range = [0, max_value]
    )
    fig.update_traces(opacity=0.75)
    fig.show()
    fig.write_image("../results/figures/histogram_{}_{}.png".format(name, column))
# ?fig.update_layout

In [23]:
plotHistogram(dfs, "nr", 15, 
    "Histogram of complexities (set: nr, 27 items)",
    "Number of evaluated instances", "complexity"
    )

In [24]:
plotHistogram(dfs, "zr", 15, 
    "Histogram of complexities (set: zr, 27 items)",
    "Number of evaluated instances", "complexity"
    )

In [25]:
plotHistogram(dfs, "nr", 15, 
    "Histogram of time complexities (set: nr, 27 items)",
    "Seconds", "seconds"
    )

In [26]:
plotHistogram(dfs, "zr", 15, 
    "Histogram of time complexities (set: nr, 27 items)",
    "Seconds", "seconds"
    )

In [27]:
complexities = {}
for name in ["nr", "zr"]:
    complexities[name] = {}
    for method in ["brute_force", "branch_and_bound"]:
        complexities[name][method] = {}
        complexities[name][method]["n"] = [n for n in NUMBERS]
        complexities[name][method]["complexity"] = [dfs[name][n][method]["complexity"].mean() for n in NUMBERS]
        complexities[name][method]["seconds"] = [dfs[name][n][method]["seconds"].mean() for n in NUMBERS]

# display(complexities)

In [28]:
BF = "brute_force"
BAB = "branch_and_bound"

def plotComplexity(df, column="complexity"):
    # Create figure with secondary y-axis
    fig = go.Figure()
    # Add traces
    fig.add_trace(
        go.Scatter(x=df["nr"][BF]["n"], y=df["nr"][BF][column], name="Set: nr, method Brute Forece",
        line=dict(color='blue', dash="dot"))
    )

    fig.add_trace(
        go.Scatter(x=df["zr"][BF]["n"], y=df["zr"][BF][column], name="Set: zr, method Brute Forece",
        line=dict(color='red', dash="dash"))
    )

    fig.add_trace(
        go.Scatter(x=df["nr"][BAB]["n"], y=df["nr"][BAB][column], name="Set: nr, method Branch and Bound", 
        line=dict(color="blue"))
    )

    fig.add_trace(
        go.Scatter(x=df["zr"][BAB]["n"], y=df["zr"][BAB][column], name="Set: zr, method Branch and Bound",
        line=dict(color="red")),
    )


    # Add figure title
    fig.update_layout(
        title_text="Double Y Axis Example"
    )
    fig.update_layout(barmode='overlay', title_text='Histogram of complexities (set: NR, 15 items)', # title of plot
        xaxis_title_text='Number of evaluated instances', yaxis_title_text='Count', 
        bargap=0.2, # gap between bars of adjacent location coordinates
        bargroupgap=0.1 # gap between bars of the same location coordinates
    )
    # fig.update_traces(opacity=0.75)

    # Set x-axis title
    fig.update_xaxes(title_text="xaxis title")
    fig.show()

In [29]:
# display(complexities)
plotComplexity(complexities)

In [30]:
# display(complexities)
plotComplexity(complexities, "seconds")

In [31]:
# import plotly.express as px


# complexity = {
#     "n": [x for x in NUMBERS],
#     "complexity": [dfs['zr'][x]['brute_force']["complexity"].mean() for x in NUMBERS],
#     "seconds": [dfs['zr'][x]['brute_force']["seconds"].mean() for x in NUMBERS],
# }



# df_compl = pd.DataFrame(complexity)
# fig = px.line(df_compl, x="n", y="seconds", title='Life expectancy in Canada')
# fig.show()



# sum_dict = {"n": NUMBERS, "complexities": complexities}
# df_sum = pd.DataFrame(sum_dict)
# display(df_sum)

# sns.lineplot(x="n", y="complexities", data=df_sum)
# Do statistics

# dfs[20]["solutions"]

# with open("./time_spent.json", "w") as f:
#     f.write(json.dumps(TIME_SPENT))
# with open("./errors.json", "w") as f:
#     f.write(json.dumps(ERRORS))




In [32]:
# Plot results

# summary = pd.read_csv("./summary.csv", sep=" ", header=None,
                    #   names=["n", "seconds", "errors", "error lines", "path"])
# pd.read("./time_spent.json", orient="index")
# summary["seconds"] = 
# df[df.columns[0]] = pd.to_datetime(df.columns[0], format='%d.%m.%Y')
# display(summary.head())

# sns.lineplot(x="n", y="seconds", data=summary)
# sns.regplot(x="n", y="seconds", data=summary, order=2)

In [33]:
# dfs['zr'][27]['brute_force']["complexity"].mean()

In [34]:
# resToDf("./data/nr/res/N{n}.csv".format(n=15))

# TODO rewrite to writeSummary

#     with open("./summary.csv", "a") as f:
#         f.write("{} {} {} {} {}\n".format(n, time, err_num, err_lines, PATH_INS))



# mal = ~check
# df_res[mal]
# display(df_ins[484:485])
# display(df_res[484:485])
# display(df_sol[484:485])
# df_ins.iloc[485]
# display(df_res)
# display(df_sol)

# Do the computing if not done elsewhere

# INSTANCES = INSTANCES_TEMPLATE
# RESULTS = RESULTS_TEST_TEMPLATE

# for n in [4, 10, 15, 20]: #NUMBERS:
#     time = clib.solve(INSTANCES.format(n=n).encode(), RESULTS.format(n=n).encode())
#     print(f"N={n}, time: {time} seconds")

    # ADD TIME TO EVERY ROW IN C
#     with open("./summary.csv", "a") as f:
#     f.write("{} {} {} {} {}\n".format(n, time, err_num, err_lines, PATH_INS))

# Example ------------------------------
# N=4, time: 0.002436647 seconds
# N=10, time: 0.008560687 seconds
# N=15, time: 0.13911387 seconds
# N=20, time: 3.220593536 seconds
# N=22, time: 10.276774363 seconds
# N=25, time: 86.253023671 seconds
# N=27, time: 340.290631575 seconds
# N=30, time: 2415.635693286 seconds
# N=32, time: 9672.943833396 seconds


# n = 4
# dataset = "zr"
# df_ins = dfs[dataset][n]["instances"]
# df_sol = dfs[dataset][n]["solutions"]
# df_bf = dfs[dataset][n]["brute_force"]

# # df_ins["B"]
# [a for a in df_sol["optimum"].index if a not in df_ins["B"].index]
# df_sol["optimum"][df_sol["optimum"].index.duplicated()]
# err_num, err_lines = check(df_ins, df_sol, df_bf, data["brute_force"].format(n=n))
# print("BF: err_num: {}, err_lines: {}".format(err_num, err_lines))

# display(dfs[15]['brute_force'])
# fig = px.histogram(dfs[15]['brute_force'], x="complexity")

# fig = px.histogram(dfs[15]['branch_and_bound'], x="complexity")

# fig = make_subplots(rows=1, cols=2, subplot_titles=("Plot 1", "Plot 2"))
# fig.add_trace(go.Histogram(x=dfs[15]['brute_force']["complexity"]), row=1, col=1)
# fig.add_trace(go.Histogram(x=dfs[15]['branch_and_bound']["complexity"]), row=1, col=2)

